In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from config import db_password
import time
import psycopg2
import pandas as pd


In [2]:
engine = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    password = db_password,
    host = "dataviz-db.csxrf9ti2aba.us-east-2.rds.amazonaws.com",
    port = 5432)
cursor = engine.cursor()

In [3]:
#query data
print(cursor.execute("select * from information_schema.columns"))

engine.commit()

None


In [4]:
#use Pandas to see all data in table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=engine)

,table_name,column_name,data_type,table_schema
0,bigquery_hits_products,visitid,integer,public
1,bigquery_hits_products,fullvisitorid,character varying,public
2,bigquery_hits_products,datasource,character varying,public
3,bigquery_hits_products,productcouponcode,character varying,public
4,bigquery_hits_products,localproductrevenue,integer,public
5,bigquery_hits_products,productbrand,character varying,public
6,bigquery_hits_products,productrevenue,integer,public
7,bigquery_hits_products,pagepath,character varying,public
8,bigquery_hits_products,isinteraction,boolean,public
9,bigquery_hits_products,isentrance,boolean,public


In [5]:
#Pull all info into pandas dataframe from RDS dB
sql = """
SELECT *
from bigquery_hits_products
"""
bigquery_hits_df = pd.read_sql(sql,con = engine)
bigquery_hits_df.head(n = 10)
#bigquery_hits_df.columns

,fullvisitorid,visitid,visitnumber,browser,hitnumber,time,minute,hour,issecure,isentrance,...,productprice,localproductprice,productquantity,productrefundamount,localproductrefundamount,isimpression,isclick,productlistname,productlistposition,productcouponcode
0,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,4990000,4990000,NaN,None,None,True,None,Category,1,None
1,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,59990000,59990000,NaN,None,None,True,None,Category,2,None
2,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,39990000,39990000,NaN,None,None,True,None,Category,3,None
3,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,6990000,6990000,NaN,None,None,True,None,Category,4,None
4,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,1500000,1500000,NaN,None,None,True,None,Category,5,None
5,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,4990000,4990000,NaN,None,None,True,None,Category,6,None
6,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,4990000,4990000,NaN,None,None,True,None,Category,7,None
7,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,22990000,22990000,NaN,None,None,True,None,Category,8,None
8,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,37990000,37990000,NaN,None,None,True,None,Category,9,None
9,2248281639583218707,1501583974,1,Chrome,1,0,39,3,None,True,...,22990000,22990000,NaN,None,None,True,None,Category,10,None


In [6]:
#Action_type is the column we're going to want to predict.
#This column dictates what kind of action the visitor took on an item

#Drop in Machine Learning logistic regression libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [7]:
bigquery_hits_df.dtypes

fullvisitorid                object
visitid                       int64
visitnumber                   int64
browser                      object
hitnumber                     int64
time                          int64
minute                        int64
hour                          int64
issecure                     object
isentrance                   object
isexit                       object
isinteraction                  bool
action_type                  object
step                          int64
type                         object
hassocialsourcereferral      object
contentgroup1                object
contentgroup2                object
datasource                   object
pagepath                     object
productsku                   object
v2productname                object
v2productcategory            object
productvariant               object
productbrand                 object
productrevenue              float64
localproductrevenue         float64
productprice                

In [8]:
#Create a list of the categorical variables
bigquery_hits_cat = bigquery_hits_df.dtypes[bigquery_hits_df.dtypes == "object"].index.tolist()

#There are some null/blank instances of some of our columns that will need to be cleaned up in future iterations
#Lets get them removed super quickly
remove_these_columns = ['fullvisitorid','isentrance','isexit','hassocialsourcereferral',
                        'issecure','productrefundamount','localproductrefundamount',
                        'productcouponcode','productbrand','isimpression','isclick',
                        'productsku','productlistname','productvariant','productrevenue','localproductrevenue',
                        'productquantity']

#remove_these_columns = ['action_type']

bigquery_hits_cat = [elem for elem in bigquery_hits_cat if elem not in remove_these_columns]
bigquery_hits_cat.remove('action_type')



#get number of unique values per category column
bigquery_hits_df[bigquery_hits_cat].nunique()

browser               12
type                   2
contentgroup1          4
contentgroup2          9
datasource             2
pagepath             159
v2productname        231
v2productcategory     49
dtype: int64

In [9]:
#slap a OneHotEncoding on this for categorical columns
enc = OneHotEncoder(sparse=False)

#fit and transform the encoding using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(bigquery_hits_df[bigquery_hits_cat]))

#add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(bigquery_hits_cat)
encode_df.head()

,browser_Android Browser,browser_Android Webview,browser_Chrome,browser_Edge,browser_Firefox,browser_Internet Explorer,browser_Nokia Browser,browser_Opera,browser_Opera Mini,browser_Safari,...,v2productcategory_Home/Office/Notebooks & Journals/,v2productcategory_Home/Office/Office Other/,v2productcategory_Home/Office/Writing Instruments/,v2productcategory_Home/Shop by Brand/,v2productcategory_Home/Shop by Brand/Android/,v2productcategory_Home/Shop by Brand/Google/,v2productcategory_Home/Shop by Brand/YouTube/,v2productcategory_Notebooks & Journals,v2productcategory_Office,v2productcategory_Wearables/Men's T-Shirts/
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
bigquery_hits_df.head()
(bigquery_hits_cat + remove_these_columns)

['browser',
 'type',
 'contentgroup1',
 'contentgroup2',
 'datasource',
 'pagepath',
 'v2productname',
 'v2productcategory',
 'fullvisitorid',
 'isentrance',
 'isexit',
 'hassocialsourcereferral',
 'issecure',
 'productrefundamount',
 'localproductrefundamount',
 'productcouponcode',
 'productbrand',
 'isimpression',
 'isclick',
 'productsku',
 'productlistname',
 'productvariant',
 'productrevenue',
 'localproductrevenue',
 'productquantity']

In [11]:
#merge one-hot encoded features and drop the originals
bigquery_hits_df1 = bigquery_hits_df.merge(encode_df,left_index = True, right_index = True)
bigquery_hits_df1 = bigquery_hits_df1.drop(columns = bigquery_hits_cat+remove_these_columns)
#bigquery_hits_df1 = bigquery_hits_df1.drop(columns = remove_these_columns)
bigquery_hits_df1.columns[20:40]
#bigquery_hits_df1.productprice

Index(['browser_Opera Mini', 'browser_Safari', 'browser_Safari (in-app)',
       'browser_UC Browser', 'type_EVENT', 'type_PAGE',
       'contentgroup1_(not set)', 'contentgroup1_Android',
       'contentgroup1_Google', 'contentgroup1_YouTube',
       'contentgroup2_(not set)', 'contentgroup2_Accessories',
       'contentgroup2_Apparel', 'contentgroup2_Bags', 'contentgroup2_Brands',
       'contentgroup2_Drinkware', 'contentgroup2_Electronics',
       'contentgroup2_Nest', 'contentgroup2_Office', 'datasource_(not set)'],
      dtype='object')

In [12]:
#Remove Action type from our features dataset
y = bigquery_hits_df1.action_type

X = bigquery_hits_df1.drop(columns =['action_type'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
#Standar scaler time lets get it

#create a StandardScaler instance
scaler = StandardScaler()

#Fit the StandardScaler
X_scaler = scaler.fit(X_train)



In [14]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=1000)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.990
